In [1]:
!pip install CatBoost

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/test_weights.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv
/kaggle/input/rohlik-sales-forecasting-challenge-v2/solution.csv


In [2]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.offline as pyo
import plotly.express as px
import seaborn as sns
from sklearn.model_selection import train_test_split,KFold
from catboost import CatBoostRegressor
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

In [4]:
train=pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_train.csv")
test=pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv")
calendar=pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/calendar.csv")
inventory=pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/inventory.csv")


In [3]:
train=pd.read_csv("D:\Kaggle competition\Rohlik Sales Prediction\sales_train.csv")
test=pd.read_csv("D:\Kaggle competition\Rohlik Sales Prediction\sales_test.csv")
calendar=pd.read_csv("D:\Kaggle competition\Rohlik Sales Prediction\calendar.csv")
inventory=pd.read_csv("D:\Kaggle competition\Rohlik Sales Prediction\inventory.csv")

## Data check

In [4]:
def data_check_train():
    print('Train\n',train.head())
    print('Train info\n',train.info())
    print('Train shape\n',train.shape)
def data_check_test():
    print('Test\n',test.head())
    print('Test info\n',test.info())
    print('Test shape\n',test.shape)
def data_check_calendar():
    print('Calendar\n',calendar.head())
    print('Calendar info\n',calendar.info())
    print('Calendar shape\n',calendar.shape)
def data_check_inventory():
    print('Inventory\n',inventory.head())
    print('Inventory info\n',inventory.info())
    print('Inventory shape\n',inventory.shape)

In [7]:
data_check_train()

Train
    unique_id        date   warehouse  total_orders  sales  sell_price_main  \
0       4845  2024-03-10  Budapest_1        6436.0  16.34           646.26   
1       4845  2021-05-25  Budapest_1        4663.0  12.63           455.96   
2       4845  2021-12-20  Budapest_1        6507.0  34.55           455.96   
3       4845  2023-04-29  Budapest_1        5463.0  34.52           646.26   
4       4845  2022-04-01  Budapest_1        5997.0  35.92           486.41   

   availability  type_0_discount  type_1_discount  type_2_discount  \
0          1.00          0.00000              0.0              0.0   
1          1.00          0.00000              0.0              0.0   
2          1.00          0.00000              0.0              0.0   
3          0.96          0.20024              0.0              0.0   
4          1.00          0.00000              0.0              0.0   

   type_3_discount  type_4_discount  type_5_discount  type_6_discount  
0              0.0          0.1

In [8]:
data_check_test()

Test
    unique_id        date warehouse  total_orders  sell_price_main  \
0       1226  2024-06-03    Brno_1        8679.0            13.13   
1       1226  2024-06-11    Brno_1        8795.0            13.13   
2       1226  2024-06-13    Brno_1       10009.0            13.13   
3       1226  2024-06-15    Brno_1        8482.0            13.13   
4       1226  2024-06-09    Brno_1        8195.0            13.13   

   type_0_discount  type_1_discount  type_2_discount  type_3_discount  \
0          0.00000              0.0              0.0              0.0   
1          0.15873              0.0              0.0              0.0   
2          0.15873              0.0              0.0              0.0   
3          0.15873              0.0              0.0              0.0   
4          0.00000              0.0              0.0              0.0   

   type_4_discount  type_5_discount  type_6_discount  
0              0.0              0.0              0.0  
1              0.0            

In [ ]:
data_check_calendar()

In [ ]:
data_check_inventory()

## Checking null values & merging dataframes

In [5]:
def check_null(df=pd.DataFrame()):
    print(df.isnull().sum())
    

In [27]:
check_null(train)

unique_id           0
date                0
warehouse           0
total_orders       52
sales              52
sell_price_main     0
availability        0
type_0_discount     0
type_1_discount     0
type_2_discount     0
type_3_discount     0
type_4_discount     0
type_5_discount     0
type_6_discount     0
dtype: int64


In [ ]:
check_null(test)

In [ ]:
check_null(calendar)

In [ ]:
check_null(inventory)

In [ ]:
calendar['holiday_name'].nunique()

In [6]:
Dict={}
for i in range(len(calendar)):
    if (calendar['holiday_name'][i]!='Nil'):
        Dict[calendar['date'][i]]=calendar['holiday_name'][i]    
train['holiday_name']=train['date'].map(Dict)       

In [7]:
Dict1={}
for i in range(len(calendar)):
     Dict1[calendar['date'][i]]=calendar['holiday'][i] 
train['holiday']=train['date'].map(Dict1)

In [8]:
Dict2={}
for i in range(len(calendar)):
     Dict2[calendar['date'][i]]=calendar['shops_closed'][i]
train['shops_closed']=train['date'].map(Dict2)


In [9]:
Dict3={}
for i in range(len(calendar)):
     Dict3[calendar['date'][i]]=calendar['winter_school_holidays'][i] 
train['winter_school_holidays']=train['date'].map(Dict3)

In [10]:
Dict4={}
for i in range(len(calendar)):
     Dict4[calendar['date'][i]]=calendar['school_holidays'][i]
train['school_holidays']=train['date'].map(Dict4)

In [11]:
Dict5={}
for i in range(len(calendar)):
     Dict5[calendar['date'][i]]=calendar['warehouse'][i]
train['warehouse_holiday']=train['date'].map(Dict5)

In [12]:
def merge_df(train=pd.DataFrame(),inventory=pd.DataFrame()):
    
    Dict6={}
    for i in range(len(inventory)):
         Dict6[inventory['unique_id'][i]]=inventory['product_unique_id'][i]
    train['product_unique_id']=train['unique_id'].map(Dict6)
    
    Dict7={}
    for i in range(len(inventory)):
         Dict7[inventory['unique_id'][i]]=inventory['name'][i]
    train['Prod_Name']=train['unique_id'].map(Dict7)
    
    Dict8={}
    for i in range(len(inventory)):
         Dict8[inventory['unique_id'][i]]=inventory['L1_category_name_en'][i]
    train['L1_category_name_en']=train['unique_id'].map(Dict8)
    
    Dict9={}
    for i in range(len(inventory)):
         Dict9[inventory['unique_id'][i]]=inventory['L2_category_name_en'][i]
    train['L2_category_name_en']=train['unique_id'].map(Dict9)
    
    Dict10={}
    for i in range(len(inventory)):
         Dict10[inventory['unique_id'][i]]=inventory['L3_category_name_en'][i]
    train['L3_category_name_en']=train['unique_id'].map(Dict10)
    
    Dict11={}
    for i in range(len(inventory)):
         Dict11[inventory['unique_id'][i]]=inventory['L4_category_name_en'][i]
    train['L4_category_name_en']=train['unique_id'].map(Dict11)
    
    

In [13]:
def merge_df1(test=pd.DataFrame(),calendar=pd.DataFrame()):
    
    Dict={}
    for i in range(len(calendar)):
        if (calendar['holiday_name'][i]!='Nil'):
            Dict[calendar['date'][i]]=calendar['holiday_name'][i]    
    test['holiday_name']=test['date'].map(Dict)       
    

    Dict1={}
    for i in range(len(calendar)):
         Dict1[calendar['date'][i]]=calendar['holiday'][i] 
    test['holiday']=test['date'].map(Dict1)
    
    Dict2={}
    for i in range(len(calendar)):
         Dict2[calendar['date'][i]]=calendar['shops_closed'][i]
    test['shops_closed']=test['date'].map(Dict2)

    
    Dict3={}
    for i in range(len(calendar)):
         Dict3[calendar['date'][i]]=calendar['winter_school_holidays'][i] 
    test['winter_school_holidays']=test['date'].map(Dict3)
    
    Dict4={}
    for i in range(len(calendar)):
         Dict4[calendar['date'][i]]=calendar['school_holidays'][i]
    test['school_holidays']=test['date'].map(Dict4)
    
    Dict5={}
    for i in range(len(calendar)):
         Dict5[calendar['date'][i]]=calendar['warehouse'][i]
    test['warehouse_holiday']=test['date'].map(Dict5)
    

In [14]:
def merge_df2(test=pd.DataFrame(),inventory=pd.DataFrame()):
    
    Dict6={}
    for i in range(len(inventory)):
         Dict6[inventory['unique_id'][i]]=inventory['product_unique_id'][i]
    test['product_unique_id']=test['unique_id'].map(Dict6)
    
    Dict7={}
    for i in range(len(inventory)):
         Dict7[inventory['unique_id'][i]]=inventory['name'][i]
    test['Prod_Name']=test['unique_id'].map(Dict7)
    
    Dict8={}
    for i in range(len(inventory)):
         Dict8[inventory['unique_id'][i]]=inventory['L1_category_name_en'][i]
    test['L1_category_name_en']=test['unique_id'].map(Dict8)
    
    Dict9={}
    for i in range(len(inventory)):
         Dict9[inventory['unique_id'][i]]=inventory['L2_category_name_en'][i]
    test['L2_category_name_en']=test['unique_id'].map(Dict9)
    
    Dict10={}
    for i in range(len(inventory)):
         Dict10[inventory['unique_id'][i]]=inventory['L3_category_name_en'][i]
    test['L3_category_name_en']=test['unique_id'].map(Dict10)
    
    Dict11={}
    for i in range(len(inventory)):
         Dict11[inventory['unique_id'][i]]=inventory['L4_category_name_en'][i]
    test['L4_category_name_en']=test['unique_id'].map(Dict11)

In [15]:
merge_df(train,inventory)

In [16]:
merge_df1(test,calendar)

In [17]:
merge_df2(test,inventory)

## Handling missing values

In [18]:
train['holiday_name'].unique()

array([nan, 'Christmas Eve',
       'Memorial Day for the Victims of the Holocaust',
       'Memorial Day for the Martyrs of Arad', 'Easter Monday',
       'Labour Day', 'Jan Hus', 'New Years Day', 'Whit sunday',
       'Cyrila a Metodej', '1st Christmas Day',
       '1848 Revolution Memorial Day (Extra holiday)', 'Den osvobozeni',
       'Memorial day of the 1956 Revolution', 'International womens day',
       'German Unity Day', 'Peace Festival in Augsburg', 'Whit monday',
       'Ascension day', "All Saints' Day Holiday",
       'Memorial Day of the Republic', 'All Saints Day',
       'Den boje za svobodu a demokracii', 'Den ceske statnosti',
       'Epiphany', '2nd Christmas Day', 'Corpus Christi',
       'Assumption of the Virgin Mary', 'Good Friday',
       'State Foundation Day',
       'Den vzniku samostatneho ceskoslovenskeho statu',
       'Memorial Day for the Victims of the Communist Dictatorships'],
      dtype=object)

In [18]:

train['holiday_name'].replace(np.nan,"Nil",inplace=True)
test['holiday_name'].replace(np.nan,"Nil",inplace=True)

In [19]:
train['total_orders'].fillna(train['total_orders'].median(),inplace=True)
train['sales'].fillna(train['sales'].median(),inplace=True)

In [22]:
check_null(train)

unique_id                 0
date                      0
warehouse                 0
total_orders              0
sales                     0
sell_price_main           0
availability              0
type_0_discount           0
type_1_discount           0
type_2_discount           0
type_3_discount           0
type_4_discount           0
type_5_discount           0
type_6_discount           0
holiday_name              0
holiday                   0
shops_closed              0
winter_school_holidays    0
school_holidays           0
warehouse_holiday         0
product_unique_id         0
Prod_Name                 0
L1_category_name_en       0
L2_category_name_en       0
L3_category_name_en       0
L4_category_name_en       0
dtype: int64


In [23]:
check_null(test)

unique_id                 0
date                      0
warehouse                 0
total_orders              0
sell_price_main           0
type_0_discount           0
type_1_discount           0
type_2_discount           0
type_3_discount           0
type_4_discount           0
type_5_discount           0
type_6_discount           0
holiday_name              0
holiday                   0
shops_closed              0
winter_school_holidays    0
school_holidays           0
warehouse_holiday         0
product_unique_id         0
Prod_Name                 0
L1_category_name_en       0
L2_category_name_en       0
L3_category_name_en       0
L4_category_name_en       0
dtype: int64


In [20]:
train['date']=pd.to_datetime(train['date'])

train['Year']=train['date'].dt.year
train['Month']=train['date'].dt.month
train['Day']=train['date'].dt.day

test['date']=pd.to_datetime(test['date'])
test['Year']=test['date'].dt.year
test['Month']=test['date'].dt.month
test['Day']=test['date'].dt.day

## Univariate Analysis

In [ ]:
sns.countplot(data=train,x='warehouse')

In [ ]:
df=train.groupby('Year').sum('total_orders')
sns.distplot(df['total_orders'])

In [ ]:
train.columns

In [ ]:
df=train.groupby('Year').sum('sell_price_main')
sns.distplot(df['sell_price_main'])

In [ ]:
df=train.groupby('Year').sum('type_0_discount')
sns.distplot(df['type_0_discount'])

In [ ]:
df=train.groupby('Year').sum('type_1_discount ')
sns.distplot(df['type_1_discount'])


In [ ]:
df=train.groupby('Year').sum('type_2_discount ')
sns.distplot(df['type_2_discount'])


In [ ]:
df=train.groupby('Year').sum('type_1_discount ')
sns.distplot(df['type_3_discount'])


In [ ]:
df=train.groupby('Year').sum('type_4_discount ')
sns.distplot(df['type_4_discount'])


In [ ]:
df=train.groupby('Year').sum('type_1_discount ')
sns.distplot(df['type_5_discount'])


In [ ]:
df=train.groupby('Year').sum('type_6_discount ')
sns.distplot(df['type_6_discount'])


## Bivariate analysis

In [ ]:
df=train.groupby('Year').sum('total_orders')
sns.lineplot(data=df,x='Year',y='total_orders')

In [ ]:
df=train.groupby('Year').sum('sell_price_main')
sns.lineplot(data=df,x='Year',y='sell_price_main')

In [ ]:
df=train.groupby('Year').sum('type_0_discount')
sns.lineplot(data=df,x='Year',y='type_0_discount')

In [ ]:
df=train.groupby('Year').sum('type_1_discount')
sns.lineplot(data=df,x='Year',y='type_1_discount')

In [ ]:
df=train.groupby('Year').sum('type_2_discount')
sns.lineplot(data=df,x='Year',y='type_2_discount')

In [ ]:
df=train.groupby('Year').sum('type_3_discount')
sns.lineplot(data=df,x='Year',y='type_3_discount')

In [ ]:
df=train.groupby('Year').sum('type_4_discount')
sns.lineplot(data=df,x='Year',y='type_4_discount')

In [ ]:
df=train.groupby('Year').sum('type_5_discount')
sns.lineplot(data=df,x='Year',y='type_5_discount')

In [ ]:
df=train.groupby('Year').sum('type_6_discount')
sns.lineplot(data=df,x='Year',y='type_6_discount')

## Feature engineering

In [21]:
train['total_orders_Q']=pd.cut(np.log1p(train['total_orders']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_0_discount_Q']=pd.cut(np.log1p(train['type_0_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_1_discount_Q']=pd.cut(np.log1p(train['type_1_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_2_discount_Q']=pd.cut(np.log1p(train['type_2_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_3_discount_Q']=pd.cut(np.log1p(train['type_3_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_4_discount_Q']=pd.cut(np.log1p(train['type_4_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_5_discount_Q']=pd.cut(np.log1p(train['type_5_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_6_discount_Q']=pd.cut(np.log1p(train['type_6_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
train['sell_price_main_Q']=pd.cut(np.log1p(train['sell_price_main']),bins=4,precision=1,labels=['S','M','L','XL'])
train['type_0_discount_Q'].fillna('XL',inplace=True)


In [22]:
test['type_0_discount_Q']=pd.cut(np.log1p(test['type_0_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['total_orders_Q']=pd.cut(np.log1p(test['total_orders']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_1_discount_Q']=pd.cut(np.log1p(test['type_1_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_2_discount_Q']=pd.cut(np.log1p(test['type_2_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_3_discount_Q']=pd.cut(np.log1p(test['type_3_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_4_discount_Q']=pd.cut(np.log1p(test['type_4_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_5_discount_Q']=pd.cut(np.log1p(test['type_5_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['type_6_discount_Q']=pd.cut(np.log1p(test['type_6_discount']),bins=4,precision=1,labels=['S','M','L','XL'])
test['sell_price_main_Q']=pd.cut(np.log1p(test['sell_price_main']),bins=4,precision=1,labels=['S','M','L','XL'])

In [23]:
Dict1={'Budapest_1':1, 'Prague_2':2, 'Brno_1':3, 'Prague_1':4, 'Prague_3':5,
       'Munich_1':6, 'Frankfurt_1':7}
train['warehoue_n']=train['warehouse'].map(Dict1)
test['warehoue_n']=test['warehouse'].map(Dict1)

In [24]:
Dict2={}
arr=['Unknown', 'Christmas Eve',
       'Memorial Day for the Victims of the Holocaust',
       'Memorial Day for the Martyrs of Arad', 'Easter Monday',
       'Labour Day', 'Jan Hus', 'New Years Day', 'Whit sunday',
       'Cyrila a Metodej', '1st Christmas Day',
       '1848 Revolution Memorial Day (Extra holiday)', 'Den osvobozeni',
       'Memorial day of the 1956 Revolution', 'International womens day',
       'German Unity Day', 'Peace Festival in Augsburg', 'Whit monday',
       'Ascension day', "All Saints' Day Holiday",
       'Memorial Day of the Republic', 'All Saints Day',
       'Den boje za svobodu a demokracii', 'Den ceske statnosti',
       'Epiphany', '2nd Christmas Day', 'Corpus Christi',
       'Assumption of the Virgin Mary', 'Good Friday',
       'State Foundation Day',
       'Den vzniku samostatneho ceskoslovenskeho statu',
       'Memorial Day for the Victims of the Communist Dictatorships']
ctr=1
for i in range(len(arr)):
    Dict2[arr[i]]=ctr
    ctr=ctr+1

    
train['holiday_name_n']=train['holiday_name'].map(Dict2)
test['holiday_name_n']=test['holiday_name'].map(Dict2)

In [25]:
Dict3={'Prague_3':1, 'Munich_1':2, 'Prague_1':3, 'Brno_1':4, 'Prague_2':5,
       'Frankfurt_1':6, 'Budapest_1':6}
train['warehouse_holiday_n']=train['warehouse_holiday'].map(Dict3)
test['warehouse_holiday_n']=test['warehouse_holiday'].map(Dict3)

In [26]:
Dict4={}
ctr=1
arr=train['Prod_Name'].unique()
for i in range(len(arr)):
    Dict4[arr[i]]=ctr
    ctr=ctr+1

    
train['Prod_name_n']=train['Prod_Name'].map(Dict4)
test['Prod_name_n']=test['Prod_Name'].map(Dict4)

In [27]:
Dict5={'Bakery':1,'Meat and fish':2,'Fruit and vegetable':3}
train['L1_category_name_en_n']=train['L1_category_name_en'].map(Dict5)
test['L1_category_name_en_n']=test['L1_category_name_en'].map(Dict5)

In [28]:
arr=['Bakery_L2_18', 'Meat and fish_L2_13', 'Fruit and vegetable_L2_3',
       'Bakery_L2_10', 'Fruit and vegetable_L2_1', 'Bakery_L2_44',
       'Bakery_L2_7', 'Bakery_L2_9', 'Fruit and vegetable_L2_11',
       'Bakery_L2_14', 'Meat and fish_L2_26', 'Bakery_L2_5',
       'Fruit and vegetable_L2_0', 'Bakery_L2_8', 'Bakery_L2_22',
       'Bakery_L2_21', 'Fruit and vegetable_L2_42', 'Meat and fish_L2_25',
       'Fruit and vegetable_L2_30', 'Meat and fish_L2_15',
       'Fruit and vegetable_L2_27', 'Fruit and vegetable_L2_2',
       'Bakery_L2_34', 'Meat and fish_L2_23', 'Fruit and vegetable_L2_37',
       'Bakery_L2_28', 'Fruit and vegetable_L2_4',
       'Fruit and vegetable_L2_12', 'Fruit and vegetable_L2_33',
       'Fruit and vegetable_L2_17', 'Fruit and vegetable_L2_6',
       'Bakery_L2_29', 'Meat and fish_L2_32', 'Fruit and vegetable_L2_41',
       'Bakery_L2_35', 'Meat and fish_L2_27', 'Bakery_L2_24',
       'Fruit and vegetable_L2_36', 'Fruit and vegetable_L2_43',
       'Bakery_L2_40', 'Meat and fish_L2_20', 'Fruit and vegetable_L2_16',
       'Fruit and vegetable_L2_31', 'Meat and fish_L2_38',
       'Fruit and vegetable_L2_39', 'Fruit and vegetable_L2_19',
       'Fruit and vegetable_L2_45']
Dict6={}
ctr=1
for i in range(len(arr)):
    Dict6[arr[i]]=ctr
    ctr=ctr+1

    
train['L2_category_name_en_n']=train['L2_category_name_en'].map(Dict6)
test['L2_category_name_en_n']=test['L2_category_name_en'].map(Dict6)

In [29]:
arr=train['L3_category_name_en'].unique()
ctr=1
Dict7={}
for i in range(len(arr)):
    Dict7[arr[i]]=ctr
    ctr=ctr+1

    
train['L3_category_name_en_n']=train['L3_category_name_en'].map(Dict7)
test['L3_category_name_en_n']=test['L3_category_name_en'].map(Dict7)

In [30]:
arr=train['L4_category_name_en'].unique()
ctr=1
Dict8={}
for i in range(len(arr)):
    Dict8[arr[i]]=ctr
    ctr=ctr+1

    
train['L4_category_name_en_n']=train['L4_category_name_en'].map(Dict8)
test['L4_category_name_en_n']=test['L4_category_name_en'].map(Dict8)

In [33]:
train_re=train.copy()
test_re=test.copy()

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 46 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  holiday_name            object        
 15  holiday                 int64         
 16  shops_closed            int64         
 17  winter_school_holidays  int64         
 18  sc

In [32]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sell_price_main         47021 non-null  float64       
 5   type_0_discount         47021 non-null  float64       
 6   type_1_discount         47021 non-null  float64       
 7   type_2_discount         47021 non-null  float64       
 8   type_3_discount         47021 non-null  float64       
 9   type_4_discount         47021 non-null  float64       
 10  type_5_discount         47021 non-null  float64       
 11  type_6_discount         47021 non-null  float64       
 12  holiday_name            47021 non-null  object

## Splitting Data

In [49]:
train.drop(columns={'date','holiday_name','warehouse_holiday','availability','warehouse','Prod_Name','L1_category_name_en','L2_category_name_en', 'L3_category_name_en','L4_category_name_en','type_0_discount_Q','total_orders_Q','type_1_discount_Q','type_2_discount_Q','type_3_discount_Q','type_4_discount_Q','type_5_discount_Q','type_6_discount_Q','sell_price_main_Q'},inplace=True)
test.drop(columns={'date','holiday_name','warehouse_holiday','warehouse','Prod_Name','L1_category_name_en','L2_category_name_en', 'L3_category_name_en','L4_category_name_en','type_0_discount_Q','total_orders_Q','type_1_discount_Q','type_2_discount_Q','type_3_discount_Q','type_4_discount_Q','type_5_discount_Q','type_6_discount_Q','sell_price_main_Q'},inplace=True)

In [40]:
print(len(train.columns))
print(len(test.columns))

27
26


In [50]:
X=train.drop(columns=['sales'])
y=train['sales']

In [51]:
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)


In [43]:
print(len(X.columns))
print(len(test.columns))

26
26


## LightGBM

In [52]:
n_splits=5
lgb_params={
    'objective':'regression',
    'metric':'MAE',
    'boosting_type':'gbdt',
    'num_leaves':20,
    'max_depth':5,
    'learning_rate':0.01,
    'n_estimators':1000,
    'early_stopping_rounds':50,
    'random_state':42
}

In [53]:
lgb_predictions=np.zeros(len(test))

kf=KFold(n_splits=n_splits,shuffle=True,random_state=42)

def acc_score(y_true,y_pred):
    return np.mean(np.abs(y_true,np.maximum(y_pred,0)))

In [54]:
for fold,(train_idx,val_idx) in enumerate(kf.split(X)):
    X_train,X_val=X.iloc[train_idx],X.iloc[val_idx]
    y_train,y_val=y.iloc[train_idx],y.iloc[val_idx]
    #lightgbm
    lgb_model=lgb.LGBMRegressor(**lgb_params)
    lgb_model.fit(X_train, np.log1p(y_train),eval_set=[(X_val, np.log1p(y_val))])
    
    lgb_val_pred=lgb_model.predict(X_val)
    print(acc_score(y_val,lgb_val_pred))
    lgb_predictions+=lgb_model.predict(test)/n_splits
    

[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.146815 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2755
[LightGBM] [Info] Number of data points in the train set: 3205935, number of used features: 26
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Start training from score 3.767853
Training until validation scores don't improve for 50 rounds
Did not meet early stopping. Best iteration is:
[1000]	valid_0's l1: 0.669412
108.89136637786902
[LightGBM] [Warning] early_stopping_round is set=50, early_stopping_rounds=50 will be ignored. Current value: early_stopping_round=50
[LightGBM] [Info] Auto-choosing 

In [55]:
lgb_predictions

array([3.53834547, 4.18894402, 4.30388845, ..., 4.90758926, 4.78553721,
       3.59584597])

In [56]:
np.exp(lgb_predictions)

array([ 34.40993965,  65.95310912,  73.98692925, ..., 135.31281716,
       119.7656853 ,  36.44651954])

## Cat Boost

In [41]:
train_re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4007419 entries, 0 to 4007418
Data columns (total 46 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   unique_id               int64         
 1   date                    datetime64[ns]
 2   warehouse               object        
 3   total_orders            float64       
 4   sales                   float64       
 5   sell_price_main         float64       
 6   availability            float64       
 7   type_0_discount         float64       
 8   type_1_discount         float64       
 9   type_2_discount         float64       
 10  type_3_discount         float64       
 11  type_4_discount         float64       
 12  type_5_discount         float64       
 13  type_6_discount         float64       
 14  holiday_name            object        
 15  holiday                 int64         
 16  shops_closed            int64         
 17  winter_school_holidays  int64         
 18  sc

In [42]:
test_re.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47021 entries, 0 to 47020
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   unique_id               47021 non-null  int64         
 1   date                    47021 non-null  datetime64[ns]
 2   warehouse               47021 non-null  object        
 3   total_orders            47021 non-null  float64       
 4   sell_price_main         47021 non-null  float64       
 5   type_0_discount         47021 non-null  float64       
 6   type_1_discount         47021 non-null  float64       
 7   type_2_discount         47021 non-null  float64       
 8   type_3_discount         47021 non-null  float64       
 9   type_4_discount         47021 non-null  float64       
 10  type_5_discount         47021 non-null  float64       
 11  type_6_discount         47021 non-null  float64       
 12  holiday_name            47021 non-null  object

In [34]:
train_re.drop(columns={'warehouse','availability','Prod_Name','warehouse_holiday','date','L1_category_name_en','L2_category_name_en', 'L3_category_name_en','L4_category_name_en'},inplace=True)

In [36]:
test_re.drop(columns={'warehouse','date','Prod_Name','warehouse_holiday',
       'L1_category_name_en','L2_category_name_en', 'L3_category_name_en','L4_category_name_en'},inplace=True)

In [37]:
print(len(train_re.columns))
print(len(test_re.columns))

37
36


In [38]:
X=train_re.drop(columns='sales')
y=train_re['sales']
X_train,X_val,y_train,y_val=train_test_split(X,y,test_size=0.2,random_state=42)

cat_features=[
 'holiday_name',
 'type_0_discount_Q','total_orders_Q','type_1_discount_Q','type_2_discount_Q','type_3_discount_Q','type_4_discount_Q',
 'type_5_discount_Q',
 'type_6_discount_Q',
 'sell_price_main_Q']

In [39]:
params={
    'iterations':600,
    'learning_rate':0.1,
    'depth':3,
    'loss_function':'MAE',
    'random_seed':42,
    'verbose':100,
    'early_stopping_rounds':50
    
}

In [40]:
cb_predictions=np.zeros(len(test_re))

In [41]:
n_splits=5
kf=KFold(n_splits=n_splits,shuffle=True,random_state=42)
fold_scores=[]
models=[]
for fold, (train_idx,val_idx) in enumerate(kf.split(X_train)):
        X_train_fold,X_val_fold=X_train.iloc[train_idx],X_train.iloc[val_idx]
        y_train_fold,y_val_fold=y_train.iloc[train_idx],y_train.iloc[val_idx]
        
        model=CatBoostRegressor(**params,cat_features=cat_features,metric_period=100)
        model.fit(X_train_fold,np.log1p(y_train_fold),use_best_model=True,eval_set=(X_val,y_val))
        
        y_pred_fold=model.predict(X_val_fold)
        
        fold_mae=np.mean(np.abs(y_val_fold-y_pred_fold))
        
        fold_scores.append(fold_mae)
        models.append(model)
        print(f"Fold{fold+1}:MAE:{fold_mae}")
        
        mean_mae=np.mean(fold_scores)
        
        print('Mean Absolute Error',mean_mae)
        cb_predictions+=model.predict(test_re)/n_splits

0:	learn: 0.9767624	test: 105.3275526	best: 105.3275526 (0)	total: 1.76s	remaining: 17m 33s
100:	learn: 0.7751551	test: 105.2454899	best: 105.2454899 (100)	total: 2m 54s	remaining: 14m 23s
200:	learn: 0.7318992	test: 105.2233605	best: 105.2233605 (200)	total: 5m 42s	remaining: 11m 20s
300:	learn: 0.7019312	test: 105.2102226	best: 105.2102226 (300)	total: 8m 32s	remaining: 8m 29s
400:	learn: 0.6787102	test: 105.2024921	best: 105.2024921 (400)	total: 11m 26s	remaining: 5m 40s
500:	learn: 0.6597515	test: 105.1960430	best: 105.1959925 (499)	total: 14m 17s	remaining: 2m 49s
599:	learn: 0.6429489	test: 105.1901642	best: 105.1901642 (599)	total: 17m 6s	remaining: 0us

bestTest = 105.1901642
bestIteration = 599

Fold1:MAE:105.1412622109219
Mean Absolute Error 105.1412622109219


0:	learn: 0.9765009	test: 105.3269747	best: 105.3269747 (0)	total: 2.45s	remaining: 24m 26s
100:	learn: 0.7748852	test: 105.2455432	best: 105.2455432 (100)	total: 2m 57s	remaining: 14m 36s
200:	learn: 0.7303676	test: 105.2232640	best: 105.2232640 (200)	total: 5m 45s	remaining: 11m 25s
300:	learn: 0.7038642	test: 105.2130694	best: 105.2130217 (299)	total: 8m 35s	remaining: 8m 32s
400:	learn: 0.6800981	test: 105.2046025	best: 105.2044977 (399)	total: 11m 24s	remaining: 5m 39s
500:	learn: 0.6605429	test: 105.1976925	best: 105.1976784 (499)	total: 14m 13s	remaining: 2m 48s
599:	learn: 0.6433195	test: 105.1919479	best: 105.1919479 (599)	total: 16m 58s	remaining: 0us

bestTest = 105.1919479
bestIteration = 599

Fold2:MAE:104.01478796699024
Mean Absolute Error 104.57802508895607


0:	learn: 0.9766142	test: 105.3271358	best: 105.3271358 (0)	total: 3s	remaining: 29m 56s
100:	learn: 0.7737808	test: 105.2467839	best: 105.2467839 (100)	total: 3m	remaining: 14m 50s
200:	learn: 0.7313299	test: 105.2258182	best: 105.2258182 (200)	total: 5m 53s	remaining: 11m 40s
300:	learn: 0.7044886	test: 105.2148670	best: 105.2148670 (300)	total: 8m 47s	remaining: 8m 43s
400:	learn: 0.6823336	test: 105.2066070	best: 105.2066070 (400)	total: 11m 40s	remaining: 5m 47s
500:	learn: 0.6616984	test: 105.1990290	best: 105.1989255 (498)	total: 14m 29s	remaining: 2m 51s
599:	learn: 0.6448059	test: 105.1930540	best: 105.1930540 (599)	total: 17m 10s	remaining: 0us

bestTest = 105.193054
bestIteration = 599

Fold3:MAE:104.70799281098375
Mean Absolute Error 104.6213476629653


0:	learn: 0.9768389	test: 105.3267090	best: 105.3267090 (0)	total: 3.46s	remaining: 34m 32s
100:	learn: 0.7752500	test: 105.2469992	best: 105.2469992 (100)	total: 2m 51s	remaining: 14m 5s
200:	learn: 0.7323413	test: 105.2241921	best: 105.2241921 (200)	total: 5m 37s	remaining: 11m 9s
300:	learn: 0.7050557	test: 105.2141879	best: 105.2141879 (300)	total: 8m 19s	remaining: 8m 15s
400:	learn: 0.6825785	test: 105.2063470	best: 105.2063470 (400)	total: 10m 59s	remaining: 5m 27s
500:	learn: 0.6632228	test: 105.1984419	best: 105.1984419 (500)	total: 13m 42s	remaining: 2m 42s
599:	learn: 0.6458103	test: 105.1930298	best: 105.1930298 (599)	total: 16m 22s	remaining: 0us

bestTest = 105.1930298
bestIteration = 599

Fold4:MAE:104.26265356613496
Mean Absolute Error 104.53167413875772


0:	learn: 0.9763874	test: 105.3276255	best: 105.3276255 (0)	total: 3.76s	remaining: 37m 33s
100:	learn: 0.7748745	test: 105.2472881	best: 105.2472881 (100)	total: 2m 53s	remaining: 14m 18s
200:	learn: 0.7312248	test: 105.2244023	best: 105.2243958 (199)	total: 5m 35s	remaining: 11m 5s
300:	learn: 0.7013376	test: 105.2111379	best: 105.2111379 (300)	total: 8m 17s	remaining: 8m 14s
400:	learn: 0.6799945	test: 105.2035806	best: 105.2035806 (400)	total: 12m 18s	remaining: 6m 6s
500:	learn: 0.6608167	test: 105.1971078	best: 105.1970945 (499)	total: 15m 1s	remaining: 2m 58s
599:	learn: 0.6457200	test: 105.1926314	best: 105.1926314 (599)	total: 17m 41s	remaining: 0us

bestTest = 105.1926314
bestIteration = 599

Fold5:MAE:104.65579595254512
Mean Absolute Error 104.5564985015152


In [42]:
cb_predictions

array([3.99773303, 4.77577325, 4.88339871, ..., 5.26582309, 4.72329975,
       3.65690411])

In [43]:
np.exp(cb_predictions)

array([ 54.47451805, 118.60198762, 132.07879887, ..., 193.60559811,
       112.53899096,  38.74121853])

In [102]:
#predictions=lgb_predictions
#predictions=cb_predictions
#predictions=(lgb_predictions+cb_predictions)/2
predictions=0.8*(lgb_predictions)+0.2*(cb_predictions)

## Submission

In [ ]:
test1=pd.read_csv("/kaggle/input/rohlik-sales-forecasting-challenge-v2/sales_test.csv")

In [103]:
test1=pd.read_csv("D:\Kaggle competition\Rohlik Sales Prediction\sales_test.csv")

In [104]:
Solution=pd.DataFrame()
Solution['id']=test1['unique_id']
for i in range(len(Solution)):
      Solution['id'][i]=str(Solution['id'][i])+'_'+str(test1['date'][i])



In [ ]:
Solution.head()

In [105]:
Solution['sales_hat']=np.exp(predictions)

In [106]:
#Solution.to_csv('D:\Kaggle competition\Rohlik Sales Prediction\Solution_lgb.csv',index=False,header=True)
#Solution.to_csv('D:\Kaggle competition\Rohlik Sales Prediction\Solution_cb.csv',index=False,header=True)
Solution.to_csv('D:\Kaggle competition\Rohlik Sales Prediction\Solution_lgbcb3_1.csv',index=False,header=True)